In [ ]:
import zhipuai
import jsonlines
import json
from tqdm import tqdm

In [ ]:
zhipuai.api_key = "your zhipu_ai api key"

In [ ]:
data = list(jsonlines.open("../data/wiki_roleplay_multilingual_test_input_w_evidence.jsonl"))

In [ ]:
results = []
for each in tqdm(data):
    message = []
    for prompt in each['prompts']:
        message.append({"role": "user", "content": prompt})
        response = zhipuai.model_api.invoke(
            model="characterglm",
           meta= {
                "user_info": "用户",
                "user_name": "用户",
                "bot_info": each['system'],
                "bot_name": each['meta']['label'],
            },
            prompt = message
        )
        if not response['success']:
            message.pop(-1)
            continue
        message.append({"role": "assistant", "content": json.loads(response['data']['choices'][0]['content']).strip()})
    each['messages'] = message
    results.append(each)

In [ ]:
for i in range(len(results)):
    results[i]['messages'] = [{'role': 'system', 'content': results[i]['system']}] + results[i]['messages']

In [ ]:
with jsonlines.open(f"../data/results/wiki_roleplay_multilingual_test_input_w_evidence_characterglm_output.jsonl", "w") as f:
    for each in results:
        f.write(each)